In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch
import math

In [119]:
#here set up the path of your file, a priori it should be on same directory
data0 = pd.read_csv("ibtracs.last3years.list.v04r00.csv") 
# Preview the first 5 lines of the loaded data 
#drop first row 
data= data0.drop(data0.index[0])
data = data.replace(' ', np.nan)#fill space with nan
data.head()

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,8,9,14,19,20,25,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,BOM_GUST_PER,REUNION_GUST,REUNION_GUST_PER,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR
1,2016005N02187,2016,1,EP,CP,PALI,2016-01-05 06:00:00,NR,2.00000,-173.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,73
2,2016005N02187,2016,1,EP,CP,PALI,2016-01-05 09:00:00,NR,2.04500,-173.353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,71
3,2016005N02187,2016,1,EP,CP,PALI,2016-01-05 12:00:00,NR,2.10000,-173.200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,67
4,2016005N02187,2016,1,EP,CP,PALI,2016-01-05 15:00:00,NR,2.17750,-173.042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,56
5,2016005N02187,2016,1,EP,CP,PALI,2016-01-05 18:00:00,NR,2.30000,-172.900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,38


In [127]:
# #count missing values along columns
# print('missing value percentages for columns=', data.isnull().sum(axis=0)/data.shape[0])
#drop columns with nan threashold > 0.6
data1 = data.dropna(axis=1, thresh=int(0.5*len(data)))
print('number of columns after dropna = ', data1.shape[1])

number of columns after dropna =  30


In [121]:
data1.columns

Index(['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME',
       'NATURE', 'LAT', 'LON', 'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'IFLAG',
       'USA_ATCF_ID', 'USA_LAT', 'USA_LON', 'USA_STATUS', 'USA_WIND',
       'USA_PRES', 'USA_SSHS', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW',
       'USA_R34_NW', 'USA_POCI', 'USA_ROCI', 'USA_RMW', 'STORM_SPEED',
       'STORM_DIR'],
      dtype='object')

In [122]:
#allows to keep only specific columns
def select_data(data):
    return data[['SID', 'BASIN','ISO_TIME', 'DIST2LAND', 'LAT', 'LON',  'USA_WIND',
       'USA_PRES', 'USA_SSHS', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW',
       'USA_R34_NW', 'USA_POCI', 'USA_ROCI', 'USA_RMW', 'STORM_SPEED',
       'STORM_DIR']]#, 'BASIN', 'NATURE']]

#convert columns to numeric values
#and interpolate missing values
def numeric_data(data):
    for i in ['DIST2LAND', 'LAT', 'LON',  'USA_WIND',
       'USA_PRES', 'USA_SSHS', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW',
       'USA_R34_NW', 'USA_POCI', 'USA_ROCI', 'USA_RMW', 'STORM_SPEED',
       'STORM_DIR']:
        data[i]=pd.to_numeric(data[i],errors='coerce').astype('float64')
        data[i]=data[i].interpolate(method='linear')
        data[i]=data[i].fillna(method='ffill')
        data[i]=data[i].fillna(method='bfill')
    return data

# df0 is  cleaned data 
df0 = select_data(data)
df0 = numeric_data(df0)
df0.dtypes

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

SID             object
BASIN           object
ISO_TIME        object
DIST2LAND      float64
LAT            float64
LON            float64
USA_WIND       float64
USA_PRES       float64
USA_SSHS       float64
USA_R34_NE     float64
USA_R34_SE     float64
USA_R34_SW     float64
USA_R34_NW     float64
USA_POCI       float64
USA_ROCI       float64
USA_RMW        float64
STORM_SPEED    float64
STORM_DIR      float64
dtype: object

In [133]:
#select storms belong to certain basin, storm them in a dictionary 
def storm_to_dict(data, basin='EP', min_steps=30, min_wind=30): 
    #create empty dictionary
    dict0={}
    ind = 0
    #groupby SID 
    grouped = data.groupby('SID')
    for name, df in grouped:
#         df.reset_index(inplace=True)#reset index 
        if df['BASIN'].iloc[0]==basin:  #select only particular basin 
            if df.shape[0]>=min_steps: #select only if enough data is present 
                df = df.loc[df['USA_WIND']>=min_wind] #keep only greater than min speed
                df = df[['DIST2LAND', 'LAT', 'LON',  'USA_WIND',
       'USA_PRES', 'USA_SSHS', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW',
       'USA_R34_NW', 'USA_POCI', 'USA_ROCI', 'USA_RMW', 'STORM_SPEED',
       'STORM_DIR']] #keep only numerical columns 
                dict0.update({ind:df})
                ind+=1
    print('number of storms in basin %s is %s' %(basin, len(dict0)))
    return dict0  


In [125]:
storm_dict[0].head()

,DIST2LAND,LAT,LON,USA_WIND,USA_PRES,USA_SSHS,USA_R34_NE,USA_R34_SE,USA_R34_SW,USA_R34_NW,USA_POCI,USA_ROCI,USA_RMW,STORM_SPEED,STORM_DIR
17,1897.0,3.40000,-171.300,30.0,1004.0,-3.0,90.0,60.0,90.0,80.0,1008.0,230.0,90.0,2.0,3.0
18,1908.0,3.51250,-171.307,30.0,1004.0,-3.0,90.0,60.0,90.0,80.0,1008.0,250.0,90.0,3.0,0.0
19,1930.0,3.70000,-171.300,30.0,1004.0,-1.0,90.0,60.0,90.0,80.0,1009.0,270.0,90.0,5.0,11.0
20,1965.0,4.02751,-171.200,37.0,1002.0,0.0,90.0,60.0,90.0,80.0,1008.0,260.0,82.0,7.0,16.0
21,2010.0,4.40000,-171.100,45.0,1000.0,0.0,90.0,60.0,90.0,80.0,1008.0,250.0,75.0,7.0,12.0


In [143]:
#calculate maximum time steps among hurricanes 
def get_tensor_shape(dict0):
    tmax= 0 
    #get the max time steps: 
    for i in dict0:
        if  dict0[i].shape[0]> tmax :
            tmax = dict0[i].shape[0]
    
    #dimensions 
    d0 = len(dict0)
    d1 = dict0[0].shape[1]
    d2=tmax
    print('tensor shape is ', [d0,d1,d2])
    return d0, d1, d2


#create a tensor
def dict_to_tensor(storm_dict):
    d0, d1, d2 = get_tensor_shape(storm_dict) #d2 = max time step among hurricanes 
    tensor = np.zeros((d0,d1,d2)) #pad with zero 
    
    for i in storm_dict:
        m = storm_dict[i]
        d2 = m.shape[0]
        tensor[i,:,:d2] = m.to_numpy().T
    
    #return list of features 
    p_list = storm_dict[0].columns.tolist()
    return tensor, p_list

In [144]:
def prepare_data(since_1980=False, basin = 'EP'):
    if since_1980 == True:
        path = "ibtracs.since1980.list.v04r00.csv"
    else:
        path = "ibtracs.last3years.list.v04r00.csv"
    #read csv 
    df= pd.read_csv(path) 
    df= df.drop(data0.index[0]) #drop first row  
    df = df.replace(' ', np.nan) #fill space with nan
    
    #drop columns with nan threashold > 0.6
    df = df.dropna(axis=1, thresh=int(0.5*len(data)))
    
    df = select_data(df) #select useful columns 
    df = numeric_data(df) #formate to numeric 
    
    #select based on basin 
    storm_dict = storm_to_dict(df, basin, min_steps=30, min_wind=30)
    
    #create tensor based on storm_dict 
    tensor, p_list  = dict_to_tensor(storm_dict)
    return tensor, p_list 

In [145]:
tensor, p_list = prepare_data()

number of storms in basin EP is 67
tensor shape is  [67, 15, 117]


In [4]:
#----- load forecast data ----- 
from scipy.io import loadmat
test=loadmat('forecast_tensor.mat')

In [6]:
test.keys()

dict_keys(['__header__', '__version__', '__globals__', 'forecast_tensor'])

In [28]:
test['forecast_tensor'][0][0].dtype

dtype([('models', 'O'), ('X', 'O'), ('X_flag', 'O'), ('nhc', 'O'), ('Y', 'O'), ('label', 'O'), ('predict', 'O'), ('time', 'O')])

In [50]:
test['forecast_tensor'][0][0]['nhc'].shape

(2, 8, 13)